# Data exploration and enrichment for supervised classification
### The Hepatocellular Carcinoma Dataset

Data Exploration

In [ ]:
import pandas as pd
import numpy as np

# Carregar os dados
data = pd.read_csv('hcc_dataset.csv')
# Verificar valores ausentes
print("\nValores Ausentes:")
print((data == "?").sum())
# Substituir valores "?" por NaN
data.replace("?", np.nan, inplace=True)


print("Variáveis Categóricas:")
non_numeric_vars = [
    "Gender",
    "Symptoms",
    "Alcohol",
    "HBsAg",
    "HBeAg",
    "HBcAb",
    "HCVAb",
    "Cirrhosis",
    "Endemic",
    "Smoking",
    "Diabetes",
    "Obesity",
    "Hemochro",
    "AHT",
    "CRI",
    "HIV",
    "NASH",
    "Varices",
    "Spleno",
    "PHT",
    "PVT",
    "Metastasis",
    "Hallmark"
]

# Substituir "sim" por 1 e "nao" por 0
data.replace({"Yes": 1, "No": 0}, inplace=True)
data.replace({"Male": 1, "Female": 0}, inplace=True)
data.replace({"Active": 0, "Selfcare": 3,"Ambulatory":2,"Restricted":1,"Disabled":4}, inplace=True)
data.replace({"None": 1, "Grade I/II": 2,"Grade III/IV":3}, inplace=True)
data.replace({"None": 1, "Mild": 2,"Moderate/Severe":3}, inplace=True)
data.replace({"Lives": 1, "Dies": 0}, inplace=True)
# Exibir informações básicas sobre os dados
print("Número de registros:", len(data))
print("Número de colunas:", len(data.columns))
print("\nInformações sobre os recursos: \n")
# Converter colunas para tipo numérico (se necessário)

print(data.info())  
# Contar o número de instâncias por classe
class_distribution = data['Class'].value_counts()

# Visualizar a distribuição de classes
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
class_distribution.plot(kind='bar', color='skyblue')
plt.title('Distribuição de Classes')
plt.xlabel('Classes')
plt.ylabel('Contagem')
plt.show()
# Contar o número de instâncias por classe na coluna 'Class'
class_count = data['Class'].value_counts()

# Exibir o número de instâncias por classe
print("Número de instâncias por classe:")
print(class_count)

# Estatísticas descritivas para todas as colunas
print("\nEstatísticas Descritivas para Todas as Colunas:")  #como pomos a parecer todas as colunas
data = data.apply(pd.to_numeric, errors='ignore')
print(data.describe(include='all'))




# Identificar outliers (por exemplo, usando boxplots)
plt.figure(figsize=(10, 6))
data.boxplot()
plt.title('Boxplot dos Recursos')
plt.xticks(rotation=45)
plt.show()




In [ ]:

#Data Iputation


from sklearn.impute import KNNImputer

# Separar variáveis categóricas e numéricas
data_categorical = data[non_numeric_vars]

data_numeric = data.drop(columns=non_numeric_vars)


# Imputação para variáveis categóricas (utilizando a moda)
data_categorical_imputed = data_categorical.fillna(data_categorical.mode().iloc[0])
print("------cat-----")
print(data_categorical_imputed)
print('-------------')
# Imputação para variáveis numéricas (utilizando o KNN)
imputer_numeric = KNNImputer(n_neighbors=15, weights="distance")
data_numeric_imputed = imputer_numeric.fit_transform(data_numeric)
data_numeric_imputed = pd.DataFrame(data_numeric_imputed, columns=data_numeric.columns)
print("----------num----")
print(data_numeric_imputed)
print("--------")
# Concatenar dados numéricos imputados com variáveis categóricas imputadas
dataimput = pd.concat([ data_categorical_imputed,data_numeric_imputed,], axis=1)

print(dataimput)




In [ ]:
#data scalling 
from sklearn import preprocessing


min_max_scaler = preprocessing.MinMaxScaler()
dataminmax = min_max_scaler.fit_transform(dataimput)
dataminmax=pd.DataFrame(dataminmax,columns=data.columns)
print(dataminmax)


In [ ]:
correlation_matrix = data.corr().abs()
correlation_matrix.style.background_gradient(cmap='coolwarm')



In [ ]:
#apagar dados redundantes 
dataminmax.drop(columns=['TP','ALT','AFP','Hallmark','Leucocytes','Spleno','HIV','Obesity','HBsAg','Alcohol','Cirrhosis','Gender'],inplace=True)

print(dataminmax)

In [ ]:
#Data Modeling (Supervised Learning):
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import statistics

# 1. Identificação do Conceito Alvo
X = dataminmax.drop(columns=['Class'])  # Atributos (features)
y = dataminmax['Class']  # Variável alvo

listarecallTree0=[]
listarecallTree1=[]
listarecallKNN0=[]
listarecallKNN1=[]
listaf1Tree0=[]
listaf1Tree1=[]
listaf1KNN0=[]
listaf1KNN1=[]
for i in range(70):
    # 2. Definição dos Conjuntos de Treinamento e Teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


    # 3. Seleção e Parametrização dos Algoritmos de Aprendizado
    # Decision Tree
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train, y_train)

    # KNN
    knn_classifier = KNeighborsClassifier()
    knn_classifier.fit(X_train, y_train)


    # 4. Avaliação do Processo de Aprendizado
    # Decision Tree
    dt_predictions = dt_classifier.predict(X_test)
    dt_accuracy = accuracy_score(y_test, dt_predictions)
    dt_report = classification_report(y_test, dt_predictions, output_dict=True)
    listarecallTree0.append(dt_report['0.0']['recall'])
    listarecallTree1.append(dt_report['1.0']['recall'])
    listaf1Tree0.append(dt_report['0.0']['f1-score'])
    listaf1Tree1.append(dt_report['1.0']['f1-score'])
    

    # KNN
    knn_predictions = knn_classifier.predict(X_test)
    knn_accuracy = accuracy_score(y_test, knn_predictions)
    knn_report = classification_report(y_test, knn_predictions, output_dict=True)
    listarecallKNN0.append(knn_report['0.0']['recall'])
    listarecallKNN1.append(knn_report['1.0']['recall'])
    listaf1KNN0.append(knn_report['0.0']['f1-score'])
    listaf1KNN1.append(knn_report['1.0']['f1-score'])

mediaRTree0 = statistics.mean(listarecallTree0)
desvio_padraoRTree0 = statistics.stdev(listarecallTree0)
print("MédiaRTree0:", mediaRTree0)
print("Desvio padrão RTree0:", desvio_padraoRTree0)
mediaRKNN0 = statistics.mean(listarecallKNN0)
desvio_padraoRKNN0 = statistics.stdev(listarecallKNN0)
print("Média RKNN0:", mediaRKNN0)
print("Desvio padrão RKNN0:", desvio_padraoRKNN0)
mediaF1Tree0 = statistics.mean(listaf1Tree0)
desvio_padraoF1Tree0 = statistics.stdev(listaf1Tree0)
print("Média f1Tree0:", mediaF1Tree0)
print("Desvio padrão f1Tree0:", desvio_padraoF1Tree0)
mediaF1KNN0 = statistics.mean(listaf1KNN0)
desvio_padraoF1KNN0 = statistics.stdev(listaf1KNN0)
print("Média F1KNN0:", mediaF1KNN0)
print("Desvio padrão F1KNN0:", desvio_padraoF1KNN0)

In [ ]:
#Data Evaluation:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Função para avaliar o modelo e retornar as métricas
def evaluate_model(y_true, y_pred):
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    return cm, precision, recall, accuracy, roc_auc

# Avaliação dos Modelos
print("\nAvaliação do Modelo Decision Tree:")
cm_dt, precision_dt, recall_dt, accuracy_dt, roc_auc_dt = evaluate_model(y_test, dt_predictions)
print("\nAvaliação do Modelo KNN:")
cm_knn, precision_knn, recall_knn, accuracy_knn, roc_auc_knn = evaluate_model(y_test, knn_predictions)

# Criar DataFrame para comparar as métricas
results = pd.DataFrame({
    "Modelo": ["Decision Tree", "KNN"],
    "Precision": [precision_dt, precision_knn],
    "Recall": [recall_dt, recall_knn],
    "Accuracy": [accuracy_dt, accuracy_knn],
    "ROC AUC": [roc_auc_dt, roc_auc_knn]
})
# Exibir a matriz de confusão como texto
print("\nMatriz de Confusão - Decision Tree:")
print(cm_dt)

print("\nMatriz de Confusão - KNN:")
print(cm_knn)

# Plotar Confusion Matrix
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.heatmap(cm_dt, annot=True, fmt="d", cmap="Blues", cbar=False, square=True)
plt.title("Confusion Matrix - Decision Tree")
plt.xlabel("Predicted")
plt.ylabel("Actual")

plt.subplot(1, 2, 2)
sns.heatmap(cm_knn, annot=True, fmt="d", cmap="Blues", cbar=False, square=True)
plt.title("Confusion Matrix - KNN")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

# Plotar Curva ROC
plt.figure(figsize=(8, 6))
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, dt_predictions)
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, knn_predictions)

plt.plot(fpr_dt, tpr_dt, label='ROC curve DT (area = %0.2f)' % roc_auc_dt)
plt.plot(fpr_knn, tpr_knn, label='ROC curve KNN (area = %0.2f)' % roc_auc_knn)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# Tabela comparativa das métricas
print("\nTabela comparativa das métricas:")
print(results)




